In [1]:
data_var = ''

In [2]:
# Parameters
data_var = "2024-04-04"


In [3]:
import pandas as pd; pd.set_option('display.max_columns', None)
import os
from pycaret.classification import *
from scripts.PreparaDataset import *
from scripts.TeoriaRetornos import *

### Funções

In [4]:
def atualiza_dia(_jogos):
    _jogos = prepara_df(_jogos)

    _jogos = _jogos.reset_index(drop=True)

    return _jogos

### Pegando os jogos do dia

In [5]:
df_today = pd.read_csv(f'jogos_do_dia/{data_var}.csv')

### Atualizando os datasets que tem jogo hoje

In [6]:
temporada = pd.read_csv('data/Entire_Season.csv')
temporada = temporada[temporada['Date'] < data_var]
temporada = pd.concat([temporada, df_today])
ligas = df_today['League'].unique().tolist()
temporada = temporada[temporada['League'].isin(ligas)]
temporada.reset_index(drop=True, inplace=True)

### Cria variáveis na temporada e nos jogos do dia

In [7]:
games = atualiza_dia(temporada)
temporada = games[games['Date'] < data_var]
os.makedirs(f'apostas_do_dia/{data_var}', exist_ok=True)

In [8]:
jogos = games[games['Date'] == data_var]

jogos = jogos.drop(columns=['Home_Pts', 'Away_Pts',
       'Back_Home', 'PL_Home', 'Back_Away',
       'PL_Away', 'Back_Over', 'PL_Over', 'PL_Under', 'Back_HA_H', 'PL_HA_H',
       'PL_HA_A', ])

jogos

,Date,League,Time,Home,Away,Odds_H,Odds_A,Over_Line,Odds_Over,Odds_Under,HA_Line,HA_Odds_H,HA_Odds_A,Fixture ID,P(H),P(A),P(O),P(U),P_Diff,Porc_Over_Home,Porc_Over_Away,CustoGolHome,CustoGolAway,Avg_CG_H,DP_CG_H,CV_CG_H,Avg_CG_A,DP_CG_A,CV_CG_A,MediaCustoGolHome,MediaCustoGolAway,Last_CG_H,Last_CG_A,Acima_Last_CG_H,Acima_Last_CG_A,Abaixo_Last_CG_H,Abaixo_Last_CG_A,CV_ML,CV_Over,CV_HA,Retornos_BH_Acu,Avg_Retornos_BH,Custo_Retorno_BH,Avg_Porc_BH_Bookie,Avg_Porc_BH_Real,Dist_Porc_BH,Retornos_BA_Acu,Avg_Retornos_BA,Custo_Retorno_BA,Avg_Porc_BA_Bookie,Avg_Porc_BA_Real,Dist_Porc_BA
8500,2024-04-04,Eua Nba,20:30,New York Knicks,Sacramento Kings,1.64,2.35,214.5,1.80,2.01,-3.5,1.95,2.71,29A4vy4T,0.609756,0.425532,0.555556,0.497512,0.035288,0.4,0.8,NaN,NaN,208.838,32.552561,0.155875,199.420,43.506172,0.218164,186.832,280.928,196.00,246.34,0.0,0.0,0.0,0.0,0.251652,0.077949,0.230644,-2.18,-0.436,-1.467890,0.614649,0.6,-0.014649,2.21,0.442,3.054299,0.473257,0.6,0.126743
8501,2024-04-04,Austrália Nbl1 Sul,06:00,Melbourne Tigers,Dandenong,2.10,1.68,169.5,1.86,1.84,1.5,1.99,1.51,drbXEzR4,0.476190,0.595238,0.537634,0.543478,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.157135,0.007644,0.193949,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8502,2024-04-04,Bulgária Nbl,13:00,Levski,Academic Plovdiv,1.74,1.99,157.5,1.85,1.85,-2.5,1.99,2.41,dxDe1hw0,0.574713,0.502513,0.540541,0.540541,0.077225,0.2,0.6,NaN,NaN,289.810,115.493660,0.398515,325.552,237.695249,0.730130,196.532,426.668,418.70,145.20,1.0,0.0,0.0,0.0,0.094786,0.000000,0.134993,-3.71,-0.742,-0.997305,0.000000,0.0,0.000000,-3.35,-0.670,-1.477612,0.326793,0.1,-0.226793
8503,2024-04-04,Eua Nit,20:00,Seton Hall,Indiana State,2.22,1.61,158.5,1.80,1.90,1.5,2.03,1.53,Sj64Djhi,0.450450,0.621118,0.555556,0.526316,0.071568,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,137.41,0.00,0.0,0.0,0.0,0.0,0.225240,0.038222,0.198626,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8504,2024-04-04,Islândia Premier League,16:15,Alftanes,Hottur,1.29,3.37,168.5,1.87,1.79,-7.5,1.80,6.25,CjY3FY9T,0.775194,0.296736,0.534759,0.558659,0.071930,0.4,0.4,NaN,NaN,135.164,37.791097,0.279594,179.160,53.592203,0.299130,143.860,220.996,184.21,219.88,0.0,0.0,0.0,0.0,0.631237,0.030912,0.781770,-0.11,-0.022,-13.181818,0.000000,0.0,0.000000,-0.84,-0.168,-14.107143,0.000000,0.0,0.000000
8505,2024-04-04,Islândia Premier League,16:15,Thorl,Keflavik,1.74,1.99,188.5,1.86,1.84,-2.5,2.01,2.40,rBtmJGA4,0.574713,0.502513,0.537634,0.543478,0.077225,0.6,1.0,NaN,NaN,162.974,54.099984,0.331955,173.098,41.747971,0.241181,148.102,220.498,137.76,237.12,0.0,0.0,0.0,0.0,0.094786,0.007644,0.125067,-0.06,-0.012,-61.666667,0.000000,0.0,0.000000,3.49,0.698,1.418338,0.000000,0.0,0.000000
8506,2024-04-04,México Cibacopa,00:15,Caballeros de Culiacan,Pioneros de Los Mochis,1.82,1.90,174.5,1.87,1.79,-1.5,1.94,2.07,2FJxRp9M,0.549451,0.526316,0.534759,0.558659,0.075766,0.0,0.0,NaN,NaN,NaN,NaN,NaN,162.656,20.237329,0.124418,0.000,0.000,130.26,197.16,0.0,0.0,0.0,0.0,0.030413,0.030912,0.045847,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8507,2024-04-04,México Cibacopa,00:15,Frayles de Guasave,Halcones de Obregon,2.10,1.68,179.5,1.87,1.79,1.5,1.97,1.52,KWa9LOWq,0.476190,0.595238,0.534759,0.558659,0.071429,0.0,0.0,NaN,NaN,NaN,NaN,NaN,156.516,38.936567,0.248771,0.000,0.000,232.00,141.96,0.0,0.0,0.0,0.0,0.157135,0.030912,0.182348,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8508,2024-04-04,Nova Zelândia Nbl,03:30,Auckland Tuatara,Taranaki Airs,1.67,2.12,174.5,1.85,1.85,-2.5,1.87,2.35,ULZY7hH5,0.598802,0.471698,0.540541,0.540541,0.070501,0.0,0.0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.000,0.000,0.00,0.00,0.0,0.0,0.0,0.0,0.167915,0.000000,0.160858,0.00,0.000,inf,0.000000,0.0,0.000000,0.00,0.000,inf,0.000000,0.0,0.000000
8509,2024-04-04,Polônia Liga De Basquete,14:30,Dabrowa Gornicza,Trefl Sopot,2.08,1.70,175.5,1.87,

## Apostas do dia

### Over v0 (LDA)

In [9]:
# Duplicando o dataset
df_over_v0 = jogos.copy()

df_over_v0.reset_index(drop=True, inplace=True)
df_over_v0.replace([np.inf, -np.inf], 0, inplace=True)

if not df_over_v0.empty:
    mdl_over_v0 = load_model('ML/over/v1/v1_over', verbose=False)
    prev = predict_model(mdl_over_v0, df_over_v0)
    prev = prev[prev['prediction_label'] ==  1]
    prev.reset_index(drop=True, inplace=True)
    
    # Filtros pós modelo

    if not prev.empty:
        prev.to_csv(f'apostas_do_dia/{data_var}/over_v0_{data_var}.csv', index=False)

        print('------------ MÉTODO OVER V0 (LDA) ------------')
        print()

        prev['Bet'] = 'Over'
        ligs = prev['League'].unique().tolist()

        display(prev[['Time', 'League', 'Home', 'Away', 'Over_Line', 'Odds_Over', 'prediction_score', 'Bet']])
        print()
    else: print('Sem jogos')
else: print('Sem jogos')

------------ MÉTODO OVER V0 (LDA) ------------



,Time,League,Home,Away,Over_Line,Odds_Over,prediction_score,Bet
0,20:30,Eua Nba,New York Knicks,Sacramento Kings,214.5,1.80,1.0,Over
1,13:00,Bulgária Nbl,Levski,Academic Plovdiv,157.5,1.85,1.0,Over
2,16:15,Islândia Premier League,Alftanes,Hottur,168.5,1.87,1.0,Over
3,16:15,Islândia Premier League,Thorl,Keflavik,188.5,1.86,1.0,Over
4,00:15,México Cibacopa,Caballeros de Culiacan,Pioneros de Los Mochis,174.5,1.87,1.0,Over
5,03:30,Nova Zelândia Nbl,Auckland Tuatara,Taranaki Airs,174.5,1.85,1.0,Over
6,15:45,Reino Unido Bbl,Leicester Riders,Bristol Flyers,177.5,1.84,1.0,Over
